# Import

In [75]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten
from keras import Model, Sequential, layers, regularizers, optimizers
from keras.callbacks import EarlyStopping
from overcome_tomorrow.utils.data import *
from overcome_tomorrow.ml_logic.preprocess import *
from overcome_tomorrow.ml_logic.model import *
from tqdm import tqdm
pd.set_option('display.max_columns', None)
from sklearn import set_config; set_config(display='diagram')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of overcome_tomorrow.ml_logic.model failed: Traceback (most recent call last):
  File "/home/nico/.pyenv/versions/3.10.6/envs/overcome-tomorrow/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/home/nico/.pyenv/versions/3.10.6/envs/overcome-tomorrow/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 455, in superreload
    module = reload(module)
  File "/home/nico/.pyenv/versions/3.10.6/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/nico/code/nicblanc/over

# Load model

In [2]:
garmin_data = merge_all_data("../raw_data/Wellness/", "../raw_data/Fitness/", "../raw_data/Aggregator/")
activities = pd.read_csv("../raw_data/activities.csv", parse_dates=["timestamp", "start_time"])

✅ ../raw_data/Wellness/ loaded
✅ Wellness cleaned
✅ ../raw_data/Fitness/ loaded
✅ Fitness cleaned


/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fitness_df["date"] = pd.to_datetime(
/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:210: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_fitness_df["date"] = pd.to_datetime(clean_fitness_df["date"])
/home/nico/code/nicblanc/overcome-tomorrow/overcome_tomorrow/utils/data.py:211: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

✅ ../raw_data/Aggregator/ loaded
✅ Aggregator cleaned
✅ Datasets merged


In [3]:
garmin_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3445 entries, 2019-08-29 to 2024-03-16
Data columns (total 52 columns):
 #   Column                                 Non-Null Count  Dtype         
---  ------                                 --------------  -----         
 0   start_sleep                            3445 non-null   datetime64[ns]
 1   end_sleep                              3445 non-null   datetime64[ns]
 2   deepSleepSeconds                       3440 non-null   float64       
 3   lightSleepSeconds                      3440 non-null   float64       
 4   remSleepSeconds                        3421 non-null   float64       
 5   awakeSleepSeconds                      3440 non-null   float64       
 6   qualityScore                           1902 non-null   float64       
 7   beginTimestamp                         3383 non-null   datetime64[ns]
 8   activityTrainingLoad                   3361 non-null   float64       
 9   activityType                           3383 n

In [4]:
garmin_data.sort_values(by=["beginTimestamp"], inplace=True)
garmin_data.dropna(subset = ["beginTimestamp"], inplace=True)
garmin_data.tail(3)

,start_sleep,end_sleep,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,qualityScore,beginTimestamp,activityTrainingLoad,activityType,aerobicTrainingEffect,aerobicTrainingEffectMessage,anaerobicTrainingEffect,anaerobicTrainingEffectMessage,avgBikeCadence,avgHr,avgPower,avgRunCadence,avgSpeed,calories,caloriesConsumed,distance,duration,maxHr,maxPower,maxRunCadence,maxSpeed,moderateIntensityMinutes,normPower,sportType,trainingEffectLabel,trainingStressScore,vigorousIntensityMinutes,totalKilocalories,activeKilocalories,totalDistanceMeters,wellnessDistanceMeters,highlyActiveSeconds,activeSeconds,floorsAscendedInMeters,floorsDescendedInMeters,restingHeartRate,minAvgHeartRate,maxAvgHeartRate,hydration_sweatLossInML,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,bodyBattery_Highest,bodyBattery_Lowest,allDayStress_awake,allDayStress_asleep
calendarDate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2024-03-15,2024-03-14 20:07:00,2024-03-15 02:47:53,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 05:25:09,144.809784,gravel_cycling,3.5,IMPACTING_TEMPO_22,0.0,NO_ANAEROBIC_BENEFIT_0,77.0,127.0,218.0,NaN,0.8657,8459.65038,NaN,7.261684e+06,8.388064e+06,142.0,418.0,NaN,1.1915,1.0,221.0,CYCLING,TEMPO,147.399994,129.0,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0
2024-03-15,2024-03-14 20:07:00,2024-03-15 02:47:53,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 10:57:38,0.604935,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,75.0,NaN,NaN,0.0000,79.61038,NaN,0.000000e+00,4.022590e+05,90.0,NaN,NaN,NaN,NaN,NaN,FITNESS_EQUIPMENT,UNKNOWN,NaN,NaN,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0
2024-03-15,2024-03-14 20:07:00,2024-03-15 02:47:53,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 11:14:42,22.125320,running,1.5,RECOVERY_5,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,126.0,319.0,84.0,0.3521,586.60280,NaN,2.337730e+05,6.639260e+05,139.0,442.0,88.0,0.4749,0.0,323.0,RUNNING,RECOVERY,NaN,10.0,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0


In [5]:
activities.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3363 entries, 0 to 3362
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype              
---  ------                           --------------  -----              
 0   timestamp                        3363 non-null   datetime64[ns, UTC]
 1   total_anaerobic_training_effect  3358 non-null   float64            
 2   enhanced_avg_speed               3361 non-null   float64            
 3   avg_stroke_distance              332 non-null    float64            
 4   total_training_effect            3358 non-null   float64            
 5   num_active_lengths               317 non-null    float64            
 6   max_running_cadence              714 non-null    float64            
 7   max_heart_rate                   3356 non-null   float64            
 8   sub_sport                        3363 non-null   object             
 9   total_descent                    2392 non-null   float64            
 10  

In [6]:
activities.sort_values(by=["start_time"], inplace=True)
activities.reset_index(drop=True, inplace=True)
activities.tail(3)

,timestamp,total_anaerobic_training_effect,enhanced_avg_speed,avg_stroke_distance,total_training_effect,num_active_lengths,max_running_cadence,max_heart_rate,sub_sport,total_descent,start_time,total_distance,pool_length,178,total_calories,max_cadence,sport,188,avg_step_length,enhanced_max_speed,205,num_lengths,206,207,pool_length_unit,normalized_power,training_load_peak,total_ascent,avg_power,total_strokes,training_stress_score,avg_cadence,avg_heart_rate,max_power,activity_id
3360,2024-03-15 07:45:23+00:00,0.0,8.657,NaN,3.5,NaN,NaN,142.0,gravel_cycling,298.0,2024-03-15 05:25:09+00:00,72616.84,NaN,1371.0,2019.0,104.0,cycling,3.0,NaN,11.915,95.0,NaN,67.0,67.0,NaN,221.0,144.809784,296.0,218.0,10737.0,147.4,77.0,127.0,418.0,nicko64@hotmail.fr_241349530713
3361,2024-03-15 11:05:33+00:00,0.0,0.000,NaN,0.0,NaN,NaN,90.0,strength_training,NaN,2024-03-15 10:57:38+00:00,0.00,NaN,39.0,19.0,NaN,training,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.604935,NaN,NaN,NaN,NaN,NaN,75.0,NaN,nicko64@hotmail.fr_241374800667
3362,2024-03-15 11:26:17+00:00,0.0,3.521,NaN,1.5,NaN,88.0,139.0,generic,13.0,2024-03-15 11:14:42+00:00,2337.73,NaN,184.0,140.0,88.0,running,1.0,1265.6,4.749,95.0,NaN,91.0,91.0,NaN,323.0,22.125320,16.0,319.0,NaN,NaN,84.0,126.0,442.0,nicko64@hotmail.fr_241377639543


# Fit Preprocessors

In [4]:
preproc_garmin_data = create_preproc_garmin_data(garmin_data)
preproc_activity = create_preproc_activity(activities)

In [112]:
preproc_garmin_data

ColumnTransformer(transformers=[('knn_transformer',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=4)),
                                                 ('robust_scaler',
                                                  RobustScaler())]),
                                 Index(['deepSleepSeconds', 'lightSleepSeconds', 'remSleepSeconds',
       'awakeSleepSeconds', 'qualityScore', 'activityTrainingLoad',
       'aerobicTrainingEffect', 'anaerobicTrainingEffect', 'avgBikeCadence',
       'avgHr', 'avgPower...
      dtype='object')),
                                ('pipe_categorical',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('one_hot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['sportType', 'trainingEffectLabel']),
                                ('cycle transform', CyclicalFeaturesSleep(),
                                 Index(['start_sleep', 'end_sleep', 'beginTimestamp'], dtype='object'))])

In [113]:
preproc_activity

InvertableColumnTransformer(transformers=[('knn_imputer',
                                           Pipeline(steps=[('knn_imputer',
                                                            KNNImputer(n_neighbors=4)),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['max_heart_rate', 'total_distance',
                                            'total_training_effect',
                                            'training_load_peak', '188', '178',
                                            '206', 'total_descent',
                                            'total_ascent', 'avg_heart_rate',
                                            'total_calories',
                                            'enhanced_avg_speed',
                                            'total_anaerobic...
                                                            SimpleImputer(strategy='most_frequent')),
                                                           ('one_hot',
                                                            OneHotEncoder(drop='if_binary',
                                                                          handle_unknown='ignore',
                                                                          sparse_output=False))]),
                                           ['sport']),
                                          ('cycle_encoder',
                                           CyclicalFeaturesActivity(),
                                           ['timestamp', 'start_time']),
                                          ('imputer 100',
                                           Pipeline(steps=[('simple_imputer',
                                                            SimpleImputer(fill_value=100,
                                                                          strategy='constant')),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['205'])])

In [5]:
preproc_garmin_data.fit(garmin_data)
preproc_activity.fit(activities)

InvertableColumnTransformer(transformers=[('knn_imputer',
                                           Pipeline(steps=[('knn_imputer',
                                                            KNNImputer(n_neighbors=4)),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['enhanced_avg_speed', '207',
                                            'max_heart_rate',
                                            'total_training_effect',
                                            'total_descent',
                                            'total_anaerobic_training_effect',
                                            'total_ascent', '188',
                                            'avg_heart_rate',
                                            'training_load_peak', '178',
                                            'enhanced_max_sp...
                                                            SimpleImputer(strategy='most_frequent')),
                                                           ('one_hot',
                                                            OneHotEncoder(drop='if_binary',
                                                                          handle_unknown='ignore',
                                                                          sparse_output=False))]),
                                           ['sport']),
                                          ('cycle_encoder',
                                           CyclicalFeaturesActivity(),
                                           ['timestamp', 'start_time']),
                                          ('imputer 100',
                                           Pipeline(steps=[('simple_imputer',
                                                            SimpleImputer(fill_value=100,
                                                                          strategy='constant')),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['205'])])

In [115]:
dump(preproc_garmin_data, open("preproc_garmin_data.pkl", "wb"))
dump(preproc_activity, open("preproc_activity.pkl", "wb"))

# Create slide windows

In [11]:
preproc_activity.transform(activities.iloc[[50]])

,knn_imputer__training_load_peak,knn_imputer__avg_heart_rate,knn_imputer__206,knn_imputer__max_heart_rate,knn_imputer__total_anaerobic_training_effect,knn_imputer__total_calories,knn_imputer__188,knn_imputer__total_distance,knn_imputer__enhanced_avg_speed,knn_imputer__total_training_effect,knn_imputer__178,knn_imputer__total_ascent,knn_imputer__enhanced_max_speed,knn_imputer__207,knn_imputer__total_descent,imputer 100__205,cat_encoder__sport_cycling,cat_encoder__sport_fitness_equipment,cat_encoder__sport_generic,cat_encoder__sport_hiking,cat_encoder__sport_kayaking,cat_encoder__sport_rock_climbing,cat_encoder__sport_running,cat_encoder__sport_swimming,cat_encoder__sport_training,cat_encoder__sport_transition,cat_encoder__sport_walking,cycle_encoder__timestamp_sin,cycle_encoder__timestamp_cos,cycle_encoder__start_time_sin,cycle_encoder__start_time_cos
50,-0.37211,-0.585859,0.805755,-0.704545,0.0,-0.614973,-1.0,-0.31803,-0.290368,-0.75,-0.705292,-0.412607,-0.319958,0.780822,-0.340909,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.258819,-0.965926,0.258819,-0.965926


In [12]:
days = 30 
avg_activities_per_day = 2 
steps = days * avg_activities_per_day # sliding window size
# Prepare the training data
X_train = []
y_train = []

for i in tqdm(range(steps, activities.shape[0])):
# for i in range(steps, steps + 5):
    activity = activities.iloc[[i]]
    activity_time = activity["start_time"][i].strftime('%Y-%m-%d %H:%M:%S')
    window_df = garmin_data[garmin_data["beginTimestamp"] < activity_time].iloc[i - steps:i]
    X_train.append(preproc_garmin_data.transform(window_df))
    # TODO y_train.append(preproc_activity.transform(activity)[0]) 
    y_train.append(preproc_activity.transform(activity)) 

X_train, y_train = np.array(X_train), np.array(y_train)

print(f"{X_train.shape = } - {y_train.shape = }")

X_train.shape = (3303, 60, 66) - y_train.shape = (3303, 1, 31)


In [22]:
y_train = y_train.reshape((y_train.shape[0], y_train.shape[2])) # To remove, see above

In [23]:
y_train.shape

(3303, 31)

# Create model

In [26]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Activation
from keras.models import load_model

In [65]:
epochs = 100

model = Sequential()
model.add(LSTM(units=50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=y_train.shape[1]))
model.compile(loss="mse", optimizer="adam", metrics=['accuracy'])


In [66]:
model.fit(X_train, y_train, batch_size = 32, epochs = epochs)
model.summary

Epoch 1/100
104/104 [==============================] - 8s 33ms/step - loss: 1.0851 - accuracy: 0.1054
Epoch 2/100
104/104 [==============================] - 3s 27ms/step - loss: 1.0408 - accuracy: 0.1471
Epoch 3/100
104/104 [==============================] - 3s 29ms/step - loss: 1.0209 - accuracy: 0.1614
Epoch 4/100
104/104 [==============================] - 3s 28ms/step - loss: 1.0093 - accuracy: 0.1596
Epoch 5/100
104/104 [==============================] - 3s 29ms/step - loss: 1.0059 - accuracy: 0.1623
Epoch 6/100
104/104 [==============================] - 3s 27ms/step - loss: 0.9939 - accuracy: 0.1810
Epoch 7/100
104/104 [==============================] - 3s 28ms/step - loss: 0.9929 - accuracy: 0.1698
Epoch 8/100
104/104 [==============================] - 3s 27ms/step - loss: 0.9793 - accuracy: 0.1759
Epoch 9/100
104/104 [==============================] - 3s 25ms/step - loss: 0.9863 - accuracy: 0.1956
Epoch 10/100
104/104 [==============================] - 3s 26ms/step - loss: 0.974

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7f33b771fd30>>

In [817]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 60, 50)            23400     
                                                                 
 dropout_4 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_5 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_5 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_6 (LSTM)               (None, 60, 50)            20200     
                                                                 
 dropout_6 (Dropout)         (None, 60, 50)            0         
                                                                 
 lstm_7 (LSTM)               (None, 50)               

In [29]:
garmin_data.shape

(3383, 52)

In [490]:
window_df = garmin_data.iloc[garmin_data.shape[0] - steps - 1:garmin_data.shape[0] - 1]
input_test = preproc_garmin_data.transform(window_df)

In [491]:
first_prediction = model.predict(np.array([input_test]))

1/1 [==============================] - 0s 51ms/step


In [492]:
first_prediction

array([[ 3.26048732e-01, -1.32820457e-01, -1.08376116e-01,
        -9.08865854e-02,  1.40819505e-01,  3.11233729e-01,
        -2.72869051e-01,  8.03724587e-01,  4.38185900e-01,
        -2.00406015e-02,  5.01094699e-01,  6.59387946e-01,
         3.36818665e-01, -1.06016859e-01,  6.52424574e-01,
        -2.16555190e+00,  3.65880966e-01, -1.04546745e-03,
        -9.07090453e-06,  2.26274505e-03,  1.68244320e-03,
        -1.03137444e-03,  1.87648207e-01,  1.68635771e-01,
         6.59881532e-03,  1.39871584e-02,  2.57512599e-01,
         3.26650262e-01, -5.19587398e-01,  4.69733834e-01,
        -3.37504923e-01]], dtype=float32)

In [493]:
# def initialize_model(input_shape):
#     model = Sequential()
#     model.add(layers.LSTM(64, input_shape=input_shape))
#     model.add(layers.Dense(input_shape[1], activation='softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#     return model

In [724]:
y_train[-1]
y_train.shape

(3303, 31)

In [495]:
model.evaluate(np.array([input_test]), np.array([y_train[-1]]))

1/1 [==============================] - 0s 53ms/step - loss: 0.5193 - accuracy: 0.0000e+00


[0.5192509293556213, 0.0]

# Inverse ?

In [754]:
from sklearn.compose import ColumnTransformer

class InvertableColumnTransformer(ColumnTransformer):
    """
    Adds an inverse transform method to the standard sklearn.compose.ColumnTransformer.
    """
    def inverse_transform(self, X):
        if isinstance(X,pd.DataFrame):
            X = X.to_numpy()

        transformer_to_cols = {}
        out = {}
        for t in self.transformers_:
            name = t[0]
            cols = t[-1]
            if name not in ("remainder", "cycle_encoder"):
                transformer_to_cols[name] = cols
                for col in cols:
                    out[col] = []
        for name, indices in self.output_indices_.items():
            transformer = self.named_transformers_.get(name, None)
            start = indices.start + 1 if indices.start != 0 else indices.start
            stop = indices.stop + 1 if indices.start != 0 else indices.stop
            
            arr = X[:, start -1 : stop - 1] if stop > X.shape[1] else X[:, start: stop] 

            if transformer in (None, "remainder", "passthrough", "drop"):
                pass

            else:
                try:
                    if isinstance(transformer, Pipeline):
                        tmp = transformer[1].inverse_transform(arr)
                        current_cols = transformer_to_cols[name]
                        for sub in tmp:
                            for col, val in zip(current_cols, sub):
                                out[col].append(val)
                    else:
                        # print(self.output_indices_)
                        transformer.inverse_transform(arr)
                except Exception as e:

                    print(e)

        return pd.DataFrame.from_dict(out)

In [796]:
def create_preproc_activity_2(activity_df):

    # get numerical data
    data_num = activity_df.select_dtypes(include=np.number)
    data_num

    # get cols numerical
    numerical_features = activity_df.select_dtypes(include=np.number).columns
    numerical_features = set(numerical_features) - set(["avg_stroke_distance",
                                                        "num_active_lengths",
                                                        "num_lengths",
                                                        "max_running_cadence",
                                                        "pool_length",
                                                        "max_cadence",
                                                        "avg_step_length",
                                                        "normalized_power",
                                                        "avg_power",
                                                        "total_strokes",
                                                        "training_stress_score",
                                                        "avg_cadence",
                                                        "max_power"
                                                        ])
    numerical_features = list(numerical_features)

    # Stamina features
    # 205: beginning
    # 206: ending
    # 207: min
    beginning_stamina_features = ["205"]
    numerical_features = list(
        set(numerical_features) - set(beginning_stamina_features))

    # pipeline knn imputer
    pipe_knn_imputer = Pipeline([
        ('knn_imputer', KNNImputer(n_neighbors=4)),
        ('robust_scaler', RobustScaler())
    ])

    # pipeline simple imputer fill value with 100 for beginning stamina (205)
    pipe_nan_to_100 = Pipeline([
        ('simple_imputer', SimpleImputer(strategy="constant", fill_value=100)),
        ('robust_scaler', RobustScaler())
    ])

    # get data and cols categorial feature
    data_cat = activity_df.select_dtypes(exclude=np.number)
    cat_features = data_cat.columns
    cat_features = list(set(cat_features) - set(["pool_length_unit"]))

    # pipeline categorical

    pipe_onehot = Pipeline([
        ("simple_imputer", SimpleImputer(strategy="most_frequent")),
        ("one_hot", OneHotEncoder(sparse_output=False,
         handle_unknown="ignore", drop="if_binary"))
    ])

    full_proces = InvertableColumnTransformer(transformers=[
        ("knn_imputer", pipe_knn_imputer, numerical_features),
        ("cat_encoder", pipe_onehot, ["sport"]),
        ("cycle_encoder", CyclicalFeaturesActivity(),
         ["timestamp", "start_time"]),
        ("imputer 100", pipe_nan_to_100, beginning_stamina_features)
    ]).set_output(transform="pandas")

    return full_proces

In [835]:
preproc = create_preproc_activity(activities)
preproc.fit(activities)
preproc

InvertableColumnTransformer(transformers=[('knn_imputer',
                                           Pipeline(steps=[('knn_imputer',
                                                            KNNImputer(n_neighbors=4)),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['training_load_peak',
                                            'avg_heart_rate', '206',
                                            'max_heart_rate',
                                            'total_anaerobic_training_effect',
                                            'total_calories', '188',
                                            'total_distance',
                                            'enhanced_avg_speed',
                                            'total_training_effect', '178',
                                            'total_ascent...
                                                            SimpleImputer(strategy='most_frequent')),
                                                           ('one_hot',
                                                            OneHotEncoder(drop='if_binary',
                                                                          handle_unknown='ignore',
                                                                          sparse_output=False))]),
                                           ['sport']),
                                          ('cycle_encoder',
                                           CyclicalFeaturesActivity(),
                                           ['timestamp', 'start_time']),
                                          ('imputer 100',
                                           Pipeline(steps=[('simple_imputer',
                                                            SimpleImputer(fill_value=100,
                                                                          strategy='constant')),
                                                           ('robust_scaler',
                                                            RobustScaler())]),
                                           ['205'])])

In [836]:
first_prediction

array([[ 3.26048732e-01, -1.32820457e-01, -1.08376116e-01,
        -9.08865854e-02,  1.40819505e-01,  3.11233729e-01,
        -2.72869051e-01,  8.03724587e-01,  4.38185900e-01,
        -2.00406015e-02,  5.01094699e-01,  6.59387946e-01,
         3.36818665e-01, -1.06016859e-01,  6.52424574e-01,
        -2.16555190e+00,  3.65880966e-01, -1.04546745e-03,
        -9.07090453e-06,  2.26274505e-03,  1.68244320e-03,
        -1.03137444e-03,  1.87648207e-01,  1.68635771e-01,
         6.59881532e-03,  1.39871584e-02,  2.57512599e-01,
         3.26650262e-01, -5.19587398e-01,  4.69733834e-01,
        -3.37504923e-01]], dtype=float32)

In [840]:
preproc.inverse_transform(first_prediction).iloc[0]

training_load_peak                    69.877296
avg_heart_rate                       117.425385
206                                   74.116966
max_heart_rate                       138.000992
total_anaerobic_training_effect         0.14082
total_calories                       725.802856
188                                    1.454262
total_distance                     21228.177734
enhanced_avg_speed                     4.383346
total_training_effect                  2.143886
178                                  764.599854
total_ascent                          93.531601
enhanced_max_speed                        6.337
207                                   73.815193
total_descent                         87.413361
sport                                   cycling
timestamp                               6:09.36
start_time                              5:14.14
205                                   99.662498
Name: 0, dtype: object

In [838]:
preproc.inverse_transform(np.array([y_train[-10]]))

,training_load_peak,avg_heart_rate,206,max_heart_rate,total_anaerobic_training_effect,total_calories,188,total_distance,enhanced_avg_speed,total_training_effect,178,total_ascent,enhanced_max_speed,207,total_descent,sport,timestamp,start_time,205
0,172.981277,147.0,44.0,164.0,0.0,888.0,3.0,15179.46,4.215,3.8,999.0,44.0,4.945,44.0,42.0,running,11:15.00,10:37.30,99.741181


In [764]:
_sin = 4.69733834e-01
_cos = -3.37504923e-01

In [515]:
np.arccos(_cos) / (2 * np.pi) * 24

7.314995547189589

In [516]:
np.arcsin(_sin) / (2 * np.pi) * 24

1.8678013675153158

In [765]:
import math
angle2 = math.atan2(_sin, _cos)  # ALWAYS USE THIS
angle2 *= 180 / math.pi
if angle2 < 0: angle2 += 360
print(angle2/24)

5.23738912903143


In [778]:
def convert_sin_cos_to_hour(s, c):
    angle = math.atan2(s, c)  # ALWAYS USE THIS
    angle *= 180 / math.pi
    if angle < 0: angle += 360
    print(angle / 24)

In [779]:

convert_sin_cos_to_hour(-1.00000000e+00, -1.83697020e-16 )

11.25


In [780]:
convert_sin_cos_to_hour(-9.65925826e-01, -2.58819045e-01)

10.625000000057799


In [772]:
   # 0.46973383 -0.33750492
convert_sin_cos_to_hour(0.32665026, -0.5195874)

6.160148252132713


In [773]:
convert_sin_cos_to_hour(0.46973383, -0.33750492)

5.2373891286091245


# Test From python code

In [162]:
create_train_and_save_model()

⌛ Loading activities data from BigQuery server... ⌛ 
✅ activities data loaded, with shape (3363, 35)
⌛ Loading garmin_data data from BigQuery server... ⌛ 
✅ garmin_data data loaded, with shape (3445, 53)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3303/3303 [03:31<00:00, 15.63it/s]


Epoch 1/100


2024-03-23 19:11:27.697989: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 49148640 exceeds 10% of free system memory.


104/104 [==============================] - 7s 26ms/step - loss: 1.0803 - accuracy: 0.1147
Epoch 2/100
104/104 [==============================] - 3s 26ms/step - loss: 1.0409 - accuracy: 0.1493
Epoch 3/100
104/104 [==============================] - 3s 27ms/step - loss: 1.0237 - accuracy: 0.1589
Epoch 4/100
104/104 [==============================] - 3s 28ms/step - loss: 1.0070 - accuracy: 0.1798
Epoch 5/100
104/104 [==============================] - 3s 28ms/step - loss: 1.0031 - accuracy: 0.1765
Epoch 6/100
104/104 [==============================] - 3s 27ms/step - loss: 0.9911 - accuracy: 0.1859
Epoch 7/100
104/104 [==============================] - 3s 26ms/step - loss: 0.9835 - accuracy: 0.1765
Epoch 8/100
104/104 [==============================] - 3s 26ms/step - loss: 0.9671 - accuracy: 0.1913
Epoch 9/100
104/104 [==============================] - 3s 27ms/step - loss: 0.9687 - accuracy: 0.1977
Epoch 10/100
104/104 [==============================] - 3s 27ms/step - loss: 0.9600 - accuracy

# Predict for given date

In [9]:
!pwd

/home/nico/code/nicblanc/overcome-tomorrow/notebooks


In [78]:
preproc_garmin_data, preproc_activity, model = load_preprocessors_and_model(model_path="../raw_data", preprocessors_path="../raw_data")

In [70]:
from datetime import datetime, timedelta
now = datetime.now()
delta = timedelta(days = 30)
date = (now - delta)#.strftime('%Y-%m-%d %H:%M:%S')
date

datetime.datetime(2024, 2, 22, 20, 8, 57, 6300)

In [74]:
get_sliding_window_for_date(garmin_data, date).iloc[-1]["beginTimestamp"]

Timestamp('2024-02-22 05:09:15+0000', tz='UTC')

In [72]:
get_sliding_window_for_date(garmin_data, date).tail(3)

,calendarDate,start_sleep,end_sleep,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,qualityScore,beginTimestamp,activityTrainingLoad,activityType,aerobicTrainingEffect,aerobicTrainingEffectMessage,anaerobicTrainingEffect,anaerobicTrainingEffectMessage,avgBikeCadence,avgHr,avgPower,avgRunCadence,avgSpeed,calories,caloriesConsumed,distance,duration,maxHr,maxPower,maxRunCadence,maxSpeed,moderateIntensityMinutes,normPower,sportType,trainingEffectLabel,trainingStressScore,vigorousIntensityMinutes,totalKilocalories,activeKilocalories,totalDistanceMeters,wellnessDistanceMeters,highlyActiveSeconds,activeSeconds,floorsAscendedInMeters,floorsDescendedInMeters,restingHeartRate,minAvgHeartRate,maxAvgHeartRate,hydration_sweatLossInML,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,bodyBattery_Highest,bodyBattery_Lowest,allDayStress_awake,allDayStress_asleep
3305,2024-02-21,2024-02-20 20:16:00+00:00,2024-02-21 02:49:00+00:00,6000.0,13020.0,4560.0,0.0,90.0,2024-02-21 17:56:29+00:00,0.610550,walking,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,59.0,NaN,52.0,0.1252,217.88104,NaN,2.104490e+05,1.680297e+06,71.0,NaN,57.0,0.4171,2.0,NaN,STEPS,UNKNOWN,NaN,0.0,3602.0,1659.0,30349.0,30349.0,7767.0,3928.0,115.939,87.645,41.0,39.0,160.0,2145.0,13.0,21.0,9.0,93.0,42.0,22.0,14.0
3306,2024-02-22,2024-02-21 20:05:00+00:00,2024-02-22 02:41:15+00:00,4380.0,14640.0,4740.0,0.0,91.0,2024-02-22 04:45:05+00:00,0.437180,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,69.0,NaN,NaN,0.0000,71.23034,NaN,0.000000e+00,4.021130e+05,100.0,NaN,NaN,NaN,NaN,NaN,TRAINING,UNKNOWN,NaN,NaN,3789.0,1851.0,79421.0,8941.0,872.0,5311.0,80.407,84.068,42.0,39.0,144.0,1894.0,13.0,18.0,9.0,95.0,35.0,20.0,14.0
3307,2024-02-22,2024-02-21 20:05:00+00:00,2024-02-22 02:41:15+00:00,4380.0,14640.0,4740.0,0.0,91.0,2024-02-22 05:09:15+00:00,129.212799,virtual_ride,3.4,IMPACTING_TEMPO_22,0.0,NO_ANAEROBIC_BENEFIT_0,74.0,124.0,220.0,NaN,0.9039,7919.13780,NaN,7.051845e+06,7.801221e+06,147.0,381.0,NaN,1.3753,4.0,221.0,CYCLING,TEMPO,136.899994,89.0,3789.0,1851.0,79421.0,8941.0,872.0,5311.0,80.407,84.068,42.0,39.0,144.0,1894.0,13.0,18.0,9.0,95.0,35.0,20.0,14.0


In [166]:
predict_for_date(garmin_data, preproc_garmin_data, preproc_activity, model, date)

1/1 [==============================] - 1s 866ms/step


,max_heart_rate,total_distance,total_training_effect,training_load_peak,188,178,206,total_descent,total_ascent,avg_heart_rate,total_calories,enhanced_avg_speed,total_anaerobic_training_effect,207,enhanced_max_speed,sport,timestamp,start_time,205
0,161.121536,10696.853516,-0.80081,69.921455,2.560166,549.053345,74.866058,70.998238,99.136627,133.802658,587.004822,3.885818,0.673912,57.031303,7.092929,cycling,10:39:36,9:53:27,72.816124


In [60]:
predict_for_last_n_days(garmin_data, preproc_garmin_data, preproc_activity, model, 30)

1/1 [==============================] - 0s 28ms/step


,max_heart_rate,total_distance,total_training_effect,training_load_peak,188,178,206,total_descent,total_ascent,avg_heart_rate,total_calories,enhanced_avg_speed,total_anaerobic_training_effect,207,enhanced_max_speed,sport,timestamp,start_time,205
0,166.394073,8890.718750,0.272615,58.751450,2.516154,752.943420,74.848923,132.864288,108.427322,134.867310,934.579956,8.258449,0.763744,63.543224,12.801803,cycling,10:22:21,9:12:59,84.890938
1,151.727814,-31.257246,2.307117,6.914112,2.248977,622.230652,69.139931,93.590958,62.742508,116.003830,730.828491,5.649906,0.186064,76.299057,9.076134,cycling,10:15:33,8:41:28,98.870026
2,163.754379,3777.704346,1.609562,28.205944,2.423476,796.056519,72.895424,139.640015,86.953461,127.252090,989.843262,8.602686,0.499410,71.768967,13.282360,cycling,9:57:12,8:22:02,95.609505
3,164.461731,4369.923828,1.352545,31.305290,2.458719,801.838074,73.185478,139.134094,89.873749,128.658859,997.409424,8.621147,0.532708,70.108719,13.020673,cycling,9:55:49,8:18:05,92.960907
4,161.543304,3671.725098,2.050171,27.918797,2.349540,724.815918,72.586151,112.209572,72.648987,125.083389,843.500610,6.604935,0.310152,74.567436,10.630401,cycling,10:29:54,8:56:35,98.659355
5,154.120468,704.048889,2.467677,10.325386,2.271683,627.629028,69.908928,88.956818,59.551632,117.496368,652.679382,5.068722,0.141155,77.273071,8.451330,cycling,11:25:42,9:56:42,99.168777
6,166.402039,4152.668457,1.663166,30.884323,2.440406,812.644897,72.944771,117.341560,65.480408,128.212967,1010.325623,6.756916,0.177191,72.146164,10.187887,cycling,10:09:17,8:15:36,96.417419
7,152.353149,-945.951599,2.373943,-0.837063,2.322015,640.096008,69.128601,102.828934,67.155190,114.981255,598.765137,6.179381,0.277576,76.611443,9.769862,cycling,11:39:33,10:03:59,97.945290
8,167.223511,1475.460571,1.274664,12.173563,2.465611,943.801392,73.550423,183.411880,94.297478,127.879539,1145.387451,11.928308,0.650419,69.750626,17.698833,cycling,9:08:17,7:03:29,95.016029
9,162.904678,10469.744141,-1.962486,67.450806,2.596361,592.662170,75.831932,82.753632,106.022133,136.024780,633.894409,5.022288,0.768122,50.027306,8.223948,cycling,10:16:54,9:14:34,65.655960


In [4]:
garmin_data, activities = get_data()

⌛ Loading activities data from BigQuery server... ⌛ 
✅ activities data loaded, with shape (3363, 35)
⌛ Loading garmin_data data from BigQuery server... ⌛ 
✅ garmin_data data loaded, with shape (3445, 53)


In [169]:
garmin_data.tail()

,calendarDate,start_sleep,end_sleep,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,qualityScore,beginTimestamp,activityTrainingLoad,activityType,aerobicTrainingEffect,aerobicTrainingEffectMessage,anaerobicTrainingEffect,anaerobicTrainingEffectMessage,avgBikeCadence,avgHr,avgPower,avgRunCadence,avgSpeed,calories,caloriesConsumed,distance,duration,maxHr,maxPower,maxRunCadence,maxSpeed,moderateIntensityMinutes,normPower,sportType,trainingEffectLabel,trainingStressScore,vigorousIntensityMinutes,totalKilocalories,activeKilocalories,totalDistanceMeters,wellnessDistanceMeters,highlyActiveSeconds,activeSeconds,floorsAscendedInMeters,floorsDescendedInMeters,restingHeartRate,minAvgHeartRate,maxAvgHeartRate,hydration_sweatLossInML,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,bodyBattery_Highest,bodyBattery_Lowest,allDayStress_awake,allDayStress_asleep
3378,2024-03-14,2024-03-13 21:05:00+00:00,2024-03-14 03:20:00+00:00,3420.0,15420.0,3660.0,0.0,79.0,2024-03-14 11:25:57+00:00,0.686172,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,75.0,NaN,NaN,0.0000,87.99042,NaN,0.000000e+00,4.022960e+05,89.0,NaN,NaN,NaN,NaN,NaN,TRAINING,UNKNOWN,NaN,NaN,4563.0,2622.0,87184.0,16435.0,5810.0,2023.0,84.226,60.632,40.0,35.0,149.0,2139.0,13.0,20.0,7.0,62.0,26.0,27.0,21.0
3379,2024-03-14,2024-03-13 21:05:00+00:00,2024-03-14 03:20:00+00:00,3420.0,15420.0,3660.0,0.0,79.0,2024-03-14 17:21:40+00:00,116.148071,running,3.3,IMPROVING_AEROBIC_BASE_8,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,142.0,338.0,84.0,0.3955,3892.52858,NaN,1.601268e+06,4.048596e+06,153.0,413.0,90.0,0.4283,0.0,339.0,RUNNING,AEROBIC_BASE,NaN,68.0,4563.0,2622.0,87184.0,16435.0,5810.0,2023.0,84.226,60.632,40.0,35.0,149.0,2139.0,13.0,20.0,7.0,62.0,26.0,27.0,21.0
3380,2024-03-15,2024-03-14 20:07:00+00:00,2024-03-15 02:47:53+00:00,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 05:25:09+00:00,144.809784,gravel_cycling,3.5,IMPACTING_TEMPO_22,0.0,NO_ANAEROBIC_BENEFIT_0,77.0,127.0,218.0,NaN,0.8657,8459.65038,NaN,7.261684e+06,8.388064e+06,142.0,418.0,NaN,1.1915,1.0,221.0,CYCLING,TEMPO,147.399994,129.0,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0
3381,2024-03-15,2024-03-14 20:07:00+00:00,2024-03-15 02:47:53+00:00,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 10:57:38+00:00,0.604935,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,75.0,NaN,NaN,0.0000,79.61038,NaN,0.000000e+00,4.022590e+05,90.0,NaN,NaN,NaN,NaN,NaN,FITNESS_EQUIPMENT,UNKNOWN,NaN,NaN,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0
3382,2024-03-15,2024-03-14 20:07:00+00:00,2024-03-15 02:47:53+00:00,2760.0,20460.0,720.0,60.0,73.0,2024-03-15 11:14:42+00:00,22.125320,running,1.5,RECOVERY_5,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,126.0,319.0,84.0,0.3521,586.60280,NaN,2.337730e+05,6.639260e+05,139.0,442.0,88.0,0.4749,0.0,323.0,RUNNING,RECOVERY,NaN,10.0,4204.0,2252.0,82879.0,10263.0,1676.0,4010.0,70.453,48.237,40.0,40.0,177.0,1594.0,13.0,19.0,8.0,78.0,25.0,26.0,17.0


In [170]:
activities.tail()

,timestamp,total_anaerobic_training_effect,enhanced_avg_speed,avg_stroke_distance,total_training_effect,num_active_lengths,max_running_cadence,max_heart_rate,sub_sport,total_descent,start_time,total_distance,pool_length,178,total_calories,max_cadence,sport,188,avg_step_length,enhanced_max_speed,205,num_lengths,206,207,pool_length_unit,normalized_power,training_load_peak,total_ascent,avg_power,total_strokes,training_stress_score,avg_cadence,avg_heart_rate,max_power,activity_id
3358,2024-03-14 11:32:43+00:00,0.0,0.000,NaN,0.0,NaN,NaN,89.0,strength_training,NaN,2024-03-14 11:25:57+00:00,0.00,NaN,53.0,21.0,NaN,training,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.686172,NaN,NaN,NaN,NaN,NaN,75.0,NaN,nicko64@hotmail.fr_241188067403
3359,2024-03-14 18:32:31+00:00,0.0,3.955,NaN,3.3,NaN,90.0,153.0,generic,52.0,2024-03-14 17:21:40+00:00,16012.68,NaN,1124.0,929.0,90.0,running,2.0,1404.1,4.283,94.0,NaN,71.0,71.0,None,339.0,116.148071,56.0,338.0,NaN,NaN,84.0,142.0,413.0,nicko64@hotmail.fr_241246052903
3360,2024-03-15 07:45:23+00:00,0.0,8.657,NaN,3.5,NaN,NaN,142.0,gravel_cycling,298.0,2024-03-15 05:25:09+00:00,72616.84,NaN,1371.0,2019.0,104.0,cycling,3.0,NaN,11.915,95.0,NaN,67.0,67.0,None,221.0,144.809784,296.0,218.0,10737.0,147.4,77.0,127.0,418.0,nicko64@hotmail.fr_241349530713
3361,2024-03-15 11:05:33+00:00,0.0,0.000,NaN,0.0,NaN,NaN,90.0,strength_training,NaN,2024-03-15 10:57:38+00:00,0.00,NaN,39.0,19.0,NaN,training,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.604935,NaN,NaN,NaN,NaN,NaN,75.0,NaN,nicko64@hotmail.fr_241374800667
3362,2024-03-15 11:26:17+00:00,0.0,3.521,NaN,1.5,NaN,88.0,139.0,generic,13.0,2024-03-15 11:14:42+00:00,2337.73,NaN,184.0,140.0,88.0,running,1.0,1265.6,4.749,95.0,NaN,91.0,91.0,None,323.0,22.125320,16.0,319.0,NaN,NaN,84.0,126.0,442.0,nicko64@hotmail.fr_241377639543


In [22]:
prediction = predict_for_date(garmin_data, preproc_garmin_data, preproc_activity, model, datetime(2024, 3, 15, 0, 0, 0))
prediction

1/1 [==============================] - 0s 25ms/step


,max_heart_rate,total_distance,total_training_effect,training_load_peak,188,178,206,total_descent,total_ascent,avg_heart_rate,total_calories,enhanced_avg_speed,total_anaerobic_training_effect,207,enhanced_max_speed,sport,timestamp,start_time,205
0,160.865479,2138.778809,1.875135,18.741255,2.34506,782.651978,72.007187,133.633362,79.812675,124.053123,976.130371,8.270152,0.403994,73.477829,12.729819,cycling,9:47:46,8:06:27,98.45932


In [7]:
get_sliding_window_for_date(garmin_data, datetime(2024, 3, 15, 0, 0, 0)).tail(3)

,calendarDate,start_sleep,end_sleep,deepSleepSeconds,lightSleepSeconds,remSleepSeconds,awakeSleepSeconds,qualityScore,beginTimestamp,activityTrainingLoad,activityType,aerobicTrainingEffect,aerobicTrainingEffectMessage,anaerobicTrainingEffect,anaerobicTrainingEffectMessage,avgBikeCadence,avgHr,avgPower,avgRunCadence,avgSpeed,calories,caloriesConsumed,distance,duration,maxHr,maxPower,maxRunCadence,maxSpeed,moderateIntensityMinutes,normPower,sportType,trainingEffectLabel,trainingStressScore,vigorousIntensityMinutes,totalKilocalories,activeKilocalories,totalDistanceMeters,wellnessDistanceMeters,highlyActiveSeconds,activeSeconds,floorsAscendedInMeters,floorsDescendedInMeters,restingHeartRate,minAvgHeartRate,maxAvgHeartRate,hydration_sweatLossInML,respiration_avgWakingRespirationValue,respiration_highestRespirationValue,respiration_lowestRespirationValue,bodyBattery_Highest,bodyBattery_Lowest,allDayStress_awake,allDayStress_asleep
3376,2024-03-13,2024-03-12 20:02:00+00:00,2024-03-13 03:57:00+00:00,2520.0,22200.0,3360.0,420.0,73.0,2024-03-13 17:43:45+00:00,125.475555,running,3.3,IMPROVING_AEROBIC_BASE_8,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,141.0,338.0,84.0,0.4022,3951.18886,NaN,1.634100e+06,4.063275e+06,159.0,431.0,91.0,0.4721,0.0,340.0,RUNNING,AEROBIC_BASE,NaN,66.0,4509.0,2576.0,87474.0,16843.0,6225.0,1744.0,86.614,58.163,39.0,38.0,150.0,2382.0,13.0,19.0,9.0,61.0,24.0,28.0,24.0
3377,2024-03-14,2024-03-13 21:05:00+00:00,2024-03-14 03:20:00+00:00,3420.0,15420.0,3660.0,0.0,79.0,2024-03-14 05:39:29+00:00,164.412140,gravel_cycling,3.7,IMPACTING_TEMPO_22,0.0,NO_ANAEROBIC_BENEFIT_0,75.0,129.0,223.0,NaN,0.8895,8183.10906,NaN,7.076639e+06,7.956113e+06,138.0,379.0,NaN,1.1785,1.0,226.0,CYCLING,TEMPO,145.199997,121.0,4563.0,2622.0,87184.0,16435.0,5810.0,2023.0,84.226,60.632,40.0,35.0,149.0,2139.0,13.0,20.0,7.0,62.0,26.0,27.0,21.0
3378,2024-03-14,2024-03-13 21:05:00+00:00,2024-03-14 03:20:00+00:00,3420.0,15420.0,3660.0,0.0,79.0,2024-03-14 11:25:57+00:00,0.686172,strength_training,0.0,NO_AEROBIC_BENEFIT_18,0.0,NO_ANAEROBIC_BENEFIT_0,NaN,75.0,NaN,NaN,0.0000,87.99042,NaN,0.000000e+00,4.022960e+05,89.0,NaN,NaN,NaN,NaN,NaN,TRAINING,UNKNOWN,NaN,NaN,4563.0,2622.0,87184.0,16435.0,5810.0,2023.0,84.226,60.632,40.0,35.0,149.0,2139.0,13.0,20.0,7.0,62.0,26.0,27.0,21.0


In [44]:
real = activities[activities["start_time"].dt.strftime('%Y-%m-%d %H:%M:%S') >= datetime(2024, 3, 15, 0, 0, 0).strftime('%Y-%m-%d %H:%M:%S')].iloc[[0]]
preproc_activity.inverse_transform(preproc_activity.transform(real))
# activities[activities["start_time"].dt.strftime('%Y-%m-%d %H:%M:%S') >= datetime(2024, 3, 15, 0, 0, 0).strftime('%Y-%m-%d %H:%M:%S')]

,max_heart_rate,total_distance,total_training_effect,training_load_peak,188,178,206,total_descent,total_ascent,avg_heart_rate,total_calories,enhanced_avg_speed,total_anaerobic_training_effect,207,enhanced_max_speed,sport,timestamp,start_time,205
0,142.0,72616.84,3.5,144.809784,3.0,1371.0,67.0,298.0,296.0,127.0,2019.0,8.657,0.0,67.0,11.915,cycling,7:45:00,5:25:00,95.0


In [51]:
pd.concat([real, prediction],  keys=['real', 'prediction']).info()


,,max_heart_rate,total_distance,total_training_effect,training_load_peak,188,178,206,total_descent,total_ascent,avg_heart_rate,total_calories,enhanced_avg_speed,total_anaerobic_training_effect,207,enhanced_max_speed,sport,timestamp,start_time,205
prediction,0,160.865479,2138.778809,1.875135,18.741255,2.34506,782.651978,72.007187,133.633362,79.812675,124.053123,976.130371,8.270152,0.403994,73.477829,12.729819,cycling,9:47:46,8:06:27,98.45932


In [148]:
s, c = convert_time_to_sin_cos(pd.to_datetime(real["timestamp"]))

In [149]:
s, c

(3360    0.896873
 Name: timestamp, dtype: float64,
 3360   -0.442289
 Name: timestamp, dtype: float64)

In [157]:
convert_sin_cos_to_hour(s, c)

'7:45:00'

In [139]:
angle = math.atan2(s, c)
angle *= 180.0 / math.pi
if angle < 0:
    angle += 360.0

In [140]:
angle

116.25

In [138]:
time = angle * 24.0 / 360.0
time

7.75

In [137]:
hours = int(time)
hours

7

In [133]:
(time*60) % 60

45.0

In [134]:
(time*3600) % 60

0.0


# Test predict VS real

In [99]:
predict_vs_real_for_date(garmin_data, activities, preproc_garmin_data, preproc_activity, model, datetime(2023, 3, 15, 0, 0, 0))

1/1 [==============================] - 0s 24ms/step


,,max_heart_rate,total_distance,total_training_effect,training_load_peak,188,178,206,total_descent,total_ascent,avg_heart_rate,total_calories,enhanced_avg_speed,total_anaerobic_training_effect,207,enhanced_max_speed,sport,timestamp,start_time,205,avg_stroke_distance,num_active_lengths,max_running_cadence,sub_sport,pool_length,max_cadence,avg_step_length,num_lengths,pool_length_unit,normalized_power,avg_power,total_strokes,training_stress_score,avg_cadence,max_power,activity_id
prediction,0,161.525406,5445.330078,1.780785,39.843784,2.31277,745.297363,72.690941,105.519165,71.301971,127.052628,981.736938,6.143762,0.267114,72.918671,9.283129,cycling,9:53:18,8:21:08,98.432899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
real,2482,143.000000,3200.000000,2.400000,51.584335,2.00000,NaN,NaN,NaN,NaN,130.000000,685.000000,0.583000,0.000000,NaN,1.111000,swimming,2023-03-15 08:12:00+00:00,2023-03-15 06:39:51+00:00,NaN,0.0,128.0,NaN,lap_swimming,25.0,NaN,NaN,128.0,metric,NaN,NaN,2310.0,NaN,25.0,NaN,nicko64@hotmail.fr_179451859519


In [143]:
predict_vs_real_for_last_n_days(garmin_data, activities, preproc_garmin_data, preproc_activity, model, 10)

1/1 [==============================] - 0s 36ms/step


max_heart_rate  total_distance  total_training_effect  \
predictions 0         166.394073     8890.718750               0.272615   
            1         151.727814      -31.257246               2.307117   
            2         163.754379     3777.704346               1.609562   
            3         164.461731     4369.923828               1.352545   
            4         161.543304     3671.725098               2.050171   
            5         154.120468      704.048889               2.467677   
            6         166.402039     4152.668457               1.663166   
            7         152.353149     -945.951599               2.373943   
            8         167.223511     1475.460571               1.274664   
            9         162.904678    10469.744141              -1.962486   
reals       3329      137.000000    75036.030000               3.300000   
            3330      168.000000    10583.710000               3.400000   
            3331       79.000000        0.000000               0.000000   
            3332      142.000000    75119.310000               3.400000   
            3333      168.000000    12034.440000               3.800000   
            3334       87.000000        0.000000               0.000000   
            3335      134.000000    70618.260000               3.400000   
            3336      167.000000    12011.180000               3.700000   
            3337       87.000000        0.000000               0.000000   
            3338      130.000000    70717.990000               3.300000   

                  training_load_peak       188          178        206  \
predictions 0              58.751450  2.516154   752.943420  74.848923   
            1               6.914112  2.248977   622.230652  69.139931   
            2              28.205944  2.423476   796.056519  72.895424   
            3              31.305290  2.458719   801.838074  73.185478   
            4              27.918797  2.349540   724.815918  72.586151   
            5              10.325386  2.271683   627.629028  69.908928   
            6              30.884323  2.440406   812.644897  72.944771   
            7              -0.837063  2.322015   640.096008  69.128601   
            8              12.173563  2.465611   943.801392  73.550423   
            9              67.450806  2.596361   592.662170  75.831932   
reals       3329          125.725967  3.000000  1620.000000  68.000000   
            3330          135.001923  2.000000   730.000000  42.000000   
            3331            0.397507  0.000000    44.000000        NaN   
            3332          132.629364  3.000000  1627.000000  67.000000   
            3333          177.403259  4.000000   771.000000  43.000000   
            3334            0.688171  0.000000    50.000000        NaN   
            3335          129.411270  2.000000   870.000000  73.000000   
            3336          161.580276  3.000000   781.000000  45.000000   
            3337            0.812927  0.000000    41.000000        NaN   
            3338          117.019867  2.000000   866.000000  72.000000   

                  total_descent  total_ascent  avg_heart_rate  total_calories  \
predictions 0        132.864288    108.427322      134.867310      934.579956   
            1         93.590958     62.742508      116.003830      730.828491   
            2        139.640015     86.953461      127.252090      989.843262   
            3        139.134094     89.873749      128.658859      997.409424   
            4        112.209572     72.648987      125.083389      843.500610   
            5         88.956818     59.551632      117.496368      652.679382   
            6        117.341560     65.480408      128.212967     1010.325623   
            7        102.828934     67.155190      114.981255      598.765137   
            8        183.411880     94.297478      127.879539     1145.387451   
            9         82.753632    106.022133      136.024780      633.894409